In [1]:
import gc
import json
import pandas as pd
import numpy as np
import seaborn as sns

from tqdm import tqdm_notebook
from glob import glob

from utils import loadpkl

%matplotlib inline

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [10]:
# load feathers
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm_notebook(files, mininterval=60)], axis=1)

# split train & test
train_df = df[df['click_mode'].notnull()]
test_df = df[df['click_mode'].isnull()]

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [11]:
# load pkls
df = loadpkl('../features/plans.pkl')
queries = loadpkl('../features/queries.pkl')
profiles = loadpkl('../features/profiles.pkl')

In [12]:
# merge
df = pd.merge(df, queries, on='sid', how='left')
df = pd.merge(df, profiles, on='pid', how='left')

del queries, profiles
gc.collect()

train_df = df[df['click_mode'].notnull()]

In [16]:
target_dummies=pd.get_dummies(train_df.click_mode.astype(int), prefix='target')

In [19]:
train_df = pd.concat([train_df, target_dummies],axis=1)

In [22]:
train_df.plans_0_transport_mode.map(train_df.groupby('plans_0_transport_mode').mean()['target_0'])

0         0.060405
1         0.046703
2         0.067123
3         0.046703
4         0.067123
5         0.127590
6         0.046703
7         0.046703
8         0.046703
9         0.092444
10        0.046703
11        0.046703
12        0.067123
13        0.091560
14        0.092444
15        0.067123
16        0.046703
17        0.127590
18        0.156717
19        0.046703
20        0.067123
21        0.046703
22        0.067123
23        0.060405
24        0.046703
25        0.091560
26        0.091560
27        0.091560
28        0.046703
29        0.046703
            ...   
491024    0.060405
491025    0.046703
491026    0.046703
491027    0.060405
491028    0.067123
491029    0.046703
491030    0.067123
491031    0.060405
491032    0.046703
491033    0.091560
491034    0.092444
491035    0.122996
491036    0.067123
491037    0.092444
491038    0.067123
491039    0.067123
491040    0.092444
491041    0.046703
491042    0.060405
491043    0.046703
491044    0.127590
491045    0.

In [23]:
target_dummies.columns.to_list()

['target_0',
 'target_1',
 'target_2',
 'target_3',
 'target_4',
 'target_5',
 'target_6',
 'target_7',
 'target_8',
 'target_9',
 'target_10',
 'target_11']